# ========================================================
# Dark URL Detection
# Adrien Manciet - Thibault Sourdeval
# ========================================================

In [ ]:
#Introduction

Ce dataset est un ensemble d'url qui sont labellisés. Si le label vaut 1, l'url est dangereux, si il vaut -1, il ne l'est pas. L'objectif sera de faire un algorithme de classification des url en apprenant sur le dataset disponible. 

# =========================
# Partie 1 : Phase d'exploration
# =========================

In [ ]:
#Code

Nous notons que les fichiers de données sont sous la forme de matrices sparse. Cela signifie que seules les valeurs non nulles sont gardées en mémoire. 
Cela permet d'épargner des erreurs de mémoire. 

Le fichier features contient des numéros de features qui sont moins souvent nuls que les autres. Ils pourraient aussi être des délimiteurs de groupe
de features appartenant à la même catégorie. 

Nous codons une fonction de prévisualisation pour mieux comprendre la structure des données en les transformant en un dataframe. Pour la suite, 
nous resterons dans le format de données initial.

**Fonction de prévisualisation** 

In [ ]:
from sklearn.datasets import load_svmlight_file
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def preview_data(day, nb_lines, nb_cols, random = True):
    # Si random est laissé tel quel, une valeur au hasard 
    # est prise pour la première ligne et la première colonne à afficher

    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)
    print(X.shape)

    if random == True:
        start_line = np.random.randint(0, len(y)-nb_lines)
        start_col = np.random.randint(0,X.shape[1])
    else : 
        start_line = int(input("Première ligne à afficher : "))
        start_col = int(input("Première colonne à afficher : "))
    
    label_list = []
    for i in range(start_col, start_col+nb_cols):
        label_list.append(i)

    X_df = pd.DataFrame(X[start_line: start_line+nb_lines, start_col: start_col+nb_cols].toarray(), columns=label_list)
    y_df = pd.DataFrame(y[start_line: start_line+nb_lines], columns=['label'])

    data = pd.concat([X_df, y_df], axis=1)
    return data

preview_data(15, 10, 10, random=True)


**Visualisations grahiques** 

In [ ]:
def scatter_plot(day, feature_x, feature_y): 
    path =f"url_svmlight/url_svmlight/Day{day}.svm"
    X, y = load_svmlight_file(path)

    X = X[0:1000,:1000].toarray()
    plt.figure()
    plt.grid(alpha=0.2)
    sc = plt.scatter(X[:,feature_x], X[:,feature_y], c=y[0:1000], cmap="viridis")
    plt.xlabel(f'Feature {feature_x}')
    plt.ylabel(f'Feature {feature_y}')
    

    cbar = plt.colorbar(sc)
    cbar.set_label("Label")
    plt.show()

scatter_plot(11,4,3)

# =========================
# Partie 2 : Feature Engineering
# =========================

In [ ]:
#Code



Nous commençons par une pca brute pour voir ce que cela peut donner. 

**PCA**

In [ ]:
from sklearn.decomposition import TruncatedSVD

X, y = load_svmlight_file('url_svmlight/url_svmlight/Day96.svm')

svd = TruncatedSVD(n_components=2)
X_svd = svd.fit_transform(X)
# print(X_svd.shape)


In [ ]:
print(svd.explained_variance_ratio_)
plt.scatter(X_svd[:1000,0], X_svd[:1000,1], c=y[:1000], cmap='coolwarm', s=10, alpha=0.6)
plt.xlabel('Composante 1')
plt.ylabel('Composante 2')
plt.title('Projection des URLs sur les 2 premières composantes SVD')
plt.colorbar(label='label')
plt.show()

In [ ]:
A = np.zeros((2,3))
print(A)

Constatant l'efficacité toute relative de cette PCA, nous décidons de retravailler sur les données d'entrées afin d'éliminer dès le départ des features à trop faible variance.
Nous remarquons que beaucoup de colonnes sont nulles sur la prévisualisation, il faut les retirer du dataset. 
La difficulté est de parcourir tous les fichiers svm. 

In [ ]:
from sklearn.feature_selection import VarianceThreshold
import os

data_dir = "url_svmlight/url_svmlight"
X_list, y_list = [], []



selector = VarianceThreshold(threshold=0.1)

for file in sorted(os.listdir(data_dir))[:10]:  # exemple sur 10 jours
    X, y = load_svmlight_file(os.path.join(data_dir, file))
    # print('passage')
    X_list.append(X)
    y_list.append(y)


kept_mask = []
for i in range(len(X_list)):
    X_reduced = selector.fit_transform(X_list[i])
    print(X_list[i].shape, "→", X_reduced.shape)
    keep_mask = selector.get_support()
    kept_mask.append(np.where(keep_mask)[0])


In [ ]:

for i in range(len(X_list)):
    for j in range(len(kept_mask[1])):
        
    print(kept_mask[i][4])
# print(np.where(keep_mask)[0])

# X_reduced_df = pd.DataFrame(X_reduced[:, :100].toarray())
# print(X_reduced_df.head())
# print(np.where(keep_mask)[0])
# X_reduced = X_reduced.toarray()
# # alignement automatique par padding
# X_all = vstack(X_list).tocsr()
# y_all = np.concatenate(y_list)



In [ ]:
from sklearn.feature_selection import VarianceThreshold
from sklearn.datasets import load_svmlight_file
import os

data_dir = "url_svmlight/url_svmlight"
X_list, y_list = [], []

max_features = 3300000  # à adapter à ton dataset

selector = VarianceThreshold(threshold=0.01)
kept_mask_list = []

# Boucle sur les fichiers
for file in sorted(os.listdir(data_dir))[:20]:  # exemple sur 10 jours
    X, y = load_svmlight_file(os.path.join(data_dir, file), n_features=max_features)
    X_list.append(X)
    y_list.append(y)

    X_reduced = selector.fit_transform(X)
    keep_mask = selector.get_support()  # booléen : True si la colonne est gardée
    kept_mask_list.append(keep_mask)

# Transformer en matrice 2D : fichiers × colonnes
kept_mask_matrix = np.array(kept_mask_list, dtype=int)

# Pourcentage de fois où chaque colonne est gardée
column_keep_percentage = kept_mask_matrix.mean(axis=0) * 100

# Créer DataFrame en ne gardant que les colonnes qui ont été sélectionnées au moins une fois
df_keep = pd.DataFrame({
    'column_index': np.arange(len(column_keep_percentage)),
    'percent_kept': column_keep_percentage
})

# Filtrer les colonnes jamais gardées
df_keep = df_keep[df_keep['percent_kept'] > 0]
# Trier par pourcentage décroissant si tu veux
# df_keep = df_keep.sort_values(by='percent_kept', ascending=False).reset_index(drop=True)

# Afficher un aperçu
print(df_keep.head(30))



In [ ]:


from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

X_reduced_scaled = StandardScaler().fit_transform(X_reduced)

pca = PCA(n_components=2)
pca.fit(X_reduced_scaled)
X_reduced_proj = pca.transform(X_reduced_scaled)
print(pca.explained_variance_ratio_)

plt.figure(figsize=(15,5))
plt.subplot(1,2,1)
plt.scatter(X_reduced[:,15], X_reduced[:,16],c = y, cmap='viridis', alpha=0.5)
plt.grid(alpha=0.2)
plt.title('Dans les coordonées de base')
plt.colorbar(label='label')

plt.subplot(1,2,2)
plt.scatter(X_reduced_proj[:,0], X_reduced_proj[:,1], c = y, cmap='viridis', alpha=0.5)
plt.xlabel('Première composante principale')
plt.ylabel('Deuxième composante principale')
plt.colorbar(label='label')
plt.grid(alpha=0.2)

plt.show()



# =========================
# Partie 3 : Phase d'apprentissage
# =========================

In [ ]:
#Code

# =========================
# Partie 4 : Tuning d'un hyperparamètre
# =========================

In [ ]:
#Code

# =========================
# Partie 5 : Conclusions
# =========================

In [ ]:
#Code